In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
training_set = pd.read_csv('UNSW_NB15_training-set.csv')
training_set.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [3]:
testing_set = pd.read_csv('UNSW_NB15_testing-set.csv')
testing_set.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [4]:
testing_set.shape

(175341, 45)

In [5]:
training_set.shape

(82332, 45)

In [6]:
training_set['attack_cat'].unique()

array(['Normal', 'Reconnaissance', 'Backdoor', 'DoS', 'Exploits',
       'Analysis', 'Fuzzers', 'Worms', 'Shellcode', 'Generic'],
      dtype=object)

In [7]:
training_set['attack_cat'].value_counts()

Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
Name: attack_cat, dtype: int64

In [8]:
binary_dict = {'Normal': 'normal',
                'Generic': 'attack',
                'Exploits': 'attack',
                'Fuzzers': 'attack',
                'DoS': 'attack',
              'Reconnaissance': 'attack',
                'Analysis': 'attack',
                'Backdoor': 'attack',
                'Shellcode': 'attack',
               'Worms': 'attack'
              }

df_train_bi = testing_set.copy()
df_test_bi = training_set.copy()

df_train_bi['attack_cat'] = df_train_bi['attack_cat'].apply(binary_dict.get)
df_test_bi['attack_cat'] = df_test_bi['attack_cat'].apply(binary_dict.get)


print('Train data class: ',df_train_bi['attack_cat'].unique())
print('Train data class: ',df_test_bi['attack_cat'].unique())

print('Train data dim: ',df_train_bi.shape)
print('Test data dim: ',df_test_bi.shape)


Train data class:  ['normal' 'attack']
Train data class:  ['normal' 'attack']
Train data dim:  (175341, 45)
Test data dim:  (82332, 45)


In [9]:
mask = (df_train_bi.dtypes == np.object)
print(df_train_bi.loc[:,mask].head())
list_cat = df_train_bi.loc[:,mask].columns.tolist()
print(list_cat)
print(df_train_bi.loc[:,mask].values)

  proto service state attack_cat
0   tcp       -   FIN     normal
1   tcp       -   FIN     normal
2   tcp       -   FIN     normal
3   tcp     ftp   FIN     normal
4   tcp       -   FIN     normal
['proto', 'service', 'state', 'attack_cat']
[['tcp' '-' 'FIN' 'normal']
 ['tcp' '-' 'FIN' 'normal']
 ['tcp' '-' 'FIN' 'normal']
 ...
 ['udp' 'dns' 'INT' 'attack']
 ['udp' 'dns' 'INT' 'attack']
 ['udp' 'dns' 'INT' 'attack']]


In [11]:
mask = (df_train_bi.dtypes != np.object)
print(df_train_bi.loc[:,mask].head())
list_num = df_train_bi.loc[:,mask].columns.tolist()
print(list_num)
print(df_train_bi.loc[:,mask].values)

   id       dur  spkts  dpkts  sbytes  dbytes       rate  sttl  dttl  \
0   1  0.121478      6      4     258     172  74.087490   252   254   
1   2  0.649902     14     38     734   42014  78.473372    62   252   
2   3  1.623129      8     16     364   13186  14.170161    62   252   
3   4  1.681642     12     12     628     770  13.677108    62   252   
4   5  0.449454     10      6     534     268  33.373826   254   252   

          sload  ...  ct_src_dport_ltm  ct_dst_sport_ltm  ct_dst_src_ltm  \
0  14158.942380  ...                 1                 1               1   
1   8395.112305  ...                 1                 1               2   
2   1572.271851  ...                 1                 1               3   
3   2740.178955  ...                 1                 1               3   
4   8561.499023  ...                 2                 1              40   

   is_ftp_login  ct_ftp_cmd  ct_flw_http_mthd  ct_src_ltm  ct_srv_dst  \
0             0           0          

In [12]:
df_train_bi.loc[:,mask].describe()

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
count,175341.000000,175341.000000,175341.000000,175341.000000,1.753410e+05,1.753410e+05,1.753410e+05,175341.000000,175341.000000,1.753410e+05,...,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000
mean,87671.000000,1.359389,20.298664,18.969591,8.844844e+03,1.492892e+04,9.540619e+04,179.546997,79.609567,7.345403e+07,...,5.383538,4.206255,8.729881,0.014948,0.014948,0.133066,6.955789,9.100758,0.015752,0.680622
std,50616.731112,6.480249,136.887597,110.258271,1.747656e+05,1.436542e+05,1.654010e+05,102.940011,110.506863,1.883574e+08,...,8.047104,5.783585,10.956186,0.126048,0.126048,0.701208,8.321493,10.756952,0.124516,0.466237
min,1.000000,0.000000,1.000000,0.000000,2.800000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,43836.000000,0.000008,2.000000,0.000000,1.140000e+02,0.000000e+00,3.278614e+01,62.000000,0.000000,1.305334e+04,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000
50%,87671.000000,0.001582,2.000000,2.000000,4.300000e+02,1.640000e+02,3.225807e+03,254.000000,29.000000,8.796748e+05,...,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,3.000000,4.000000,0.000000,1.000000
75%,131506.000000,0.668069,12.000000,10.000000,1.418000e+03,1.102000e+03,1.250000e+05,254.000000,252.000000,8.888889e+07,...,5.000000,3.000000,12.000000,0.000000,0.000000,0.000000,9.000000,12.000000,0.000000,1.000000
max,175341.000000,59.999989,9616.000000,10974.000000,1.296523e+07,1.465555e+07,1.000000e+06,255.000000,254.000000,5.988000e+09,...,51.000000,46.000000,65.000000,4.000000,4.000000,30.000000,60.000000,62.000000,1.000000,1.000000


In [13]:
#  Check whether the positive label (1) match attack categories, and whether attack categories match labelled data.

# all(iterable) returns True if all elements of the iterable are considered as true values
print(all(((df_train_bi.label == 1) & (df_train_bi.attack_cat != 'normal')) == (df_train_bi.attack_cat != 'normal')))
print(all(((df_train_bi.attack_cat != 'normal') & (df_train_bi.label == 1)) == (df_train_bi.label == 1)))

True
True


In [14]:
# number of occurrences for each attack category
df_train_bi.attack_cat.value_counts()

attack    119341
normal     56000
Name: attack_cat, dtype: int64